# Azure AI Agent Notebook

This notebook provides a foundation for building AI agents using Azure OpenAI and LangChain.


## Setup and Configuration

First, let's import the necessary libraries and load our environment variables.

In [ ]:
# Core imports
import os
import asyncio
from dotenv import load_dotenv
from typing import List, Dict, Any

# LangChain imports
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter widgets
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

# Load environment variables
load_dotenv()

print("✅ All libraries imported successfully!")

## Azure OpenAI Configuration

Configure your Azure OpenAI connection using environment variables.

In [ ]:
# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

# Verify configuration
if not all([azure_endpoint, azure_api_key, chat_deployment]):
    print("❌ Missing required Azure OpenAI configuration. Please check your .env file.")
    print("Required variables: AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, AZURE_OPENAI_CHAT_DEPLOYMENT")
else:
    print("✅ Azure OpenAI configuration loaded successfully!")
    print(f"Endpoint: {azure_endpoint}")
    print(f"Chat Deployment: {chat_deployment}")
    print(f"API Version: {azure_api_version}")

## Initialize Azure OpenAI Models

Create instances of the chat and embedding models.

In [ ]:
# Initialize Azure Chat OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version,
    deployment_name=chat_deployment,
    temperature=0.7,
    max_tokens=1000
)

# Initialize embeddings (if deployment is configured)
embeddings = None
if embedding_deployment:
    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=azure_endpoint,
        api_key=azure_api_key,
        api_version=azure_api_version,
        deployment=embedding_deployment
    )
    print("✅ Embeddings model initialized")

print("✅ Azure OpenAI models initialized successfully!")

## Test Basic Chat Functionality

Let's test our Azure OpenAI connection with a simple chat.

In [ ]:
# Test basic chat
try:
    response = llm.invoke([HumanMessage(content="Hello! Can you confirm that our Azure OpenAI connection is working?")])
    print("✅ Connection successful!")
    print(f"Response: {response.content}")
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")
    print("Please check your Azure OpenAI configuration in the .env file.")

## Create Custom Tools

Define custom tools that your agent can use.

In [ ]:
@tool
def calculate(expression: str) -> str:
    """Calculate mathematical expressions safely."""
    try:
        # Simple calculator - only allow basic operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def get_current_time() -> str:
    """Get the current date and time."""
    from datetime import datetime
    return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Initialize search tool
search = DuckDuckGoSearchRun()

# List of available tools
tools = [calculate, get_current_time, search]

print("✅ Custom tools created successfully!")
print(f"Available tools: {[tool.name for tool in tools]}")

## Create an AI Agent

Now let's create an AI agent using LangChain that can use our tools.

In [ ]:
# Create a prompt template for the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant with access to various tools. 
    Use the tools when appropriate to help answer questions and solve problems.
    Be concise but thorough in your responses."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

print("✅ AI Agent created successfully!")

## Interactive Chat Interface

Create an interactive widget for chatting with your agent.

In [ ]:
# Create interactive widgets
chat_input = widgets.Text(
    placeholder='Type your message here...',
    description='Message:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

send_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='15%')
)

clear_button = widgets.Button(
    description='Clear',
    button_style='warning',
    layout=widgets.Layout(width='15%')
)

output_area = widgets.Output()

# Chat history
chat_history = []

def send_message(b):
    with output_area:
        if chat_input.value.strip():
            user_message = chat_input.value
            print(f"🧑 User: {user_message}")
            
            try:
                # Get agent response
                response = agent_executor.invoke({
                    "input": user_message,
                    "chat_history": chat_history
                })
                
                print(f"🤖 Agent: {response['output']}")
                print("-" * 50)
                
                # Update chat history
                chat_history.extend([
                    HumanMessage(content=user_message),
                    AIMessage(content=response['output'])
                ])
                
            except Exception as e:
                print(f"❌ Error: {str(e)}")
            
            chat_input.value = ''

def clear_chat(b):
    global chat_history
    chat_history = []
    output_area.clear_output()
    with output_area:
        print("Chat cleared! 🧹")

def on_enter(change):
    if change['new'] and change['new'][-1] == '\n':
        send_message(None)

# Bind events
send_button.on_click(send_message)
clear_button.on_click(clear_chat)
chat_input.observe(on_enter, names='value')

# Display the interface
input_box = widgets.HBox([chat_input, send_button, clear_button])
chat_interface = widgets.VBox([input_box, output_area])

display(Markdown("### 🤖 Chat with your AI Agent"))
display(chat_interface)

with output_area:
    print("Welcome! Your AI agent is ready. Try asking questions or requesting calculations!")
    print("Example: 'What's 25 * 4?' or 'Search for the latest news about AI'")
    print("-" * 50)

## Your Workspace

Use the cells below for your own experiments and development.

In [ ]:
# Your code here


In [ ]:
# More space for experimentation
